In [34]:
import glob
import os

import numpy as np
import pandas as pd
import scanpy as sc
import scprep as scp
import torch
import torchvision.transforms as transforms
from PIL import ImageFile, Image
import anndata as ad
from utils import *

Image.MAX_IMAGE_PIXELS = None

## Input the path where the HER2 data is located


In [5]:
cnt_dir = 'data/her2st/ST-cnts'
img_dir = 'data/her2st/ST-imgs'
pos_dir = 'data/her2st/ST-spotfiles'
lbl_dir = 'data/her2st/ST-pat/lbl'

# Extract all sample names
names = os.listdir(cnt_dir)
names.sort()
names = [i[:2] for i in names]

In [ ]:
names

## Functions to read HER2 data


In [7]:
def get_img(name):
    pre = img_dir + '/' + name[0] + '/' + name
    fig_name = os.listdir(pre)[0]
    path = pre + '/' + fig_name
    print(path)
    im = Image.open(path)
    img = np.array(im)
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]  # remove alpha channel
    return img

def get_cnt(name):
    path = cnt_dir + '/' + name + '.tsv'
    df = pd.read_csv(path, sep='\t', index_col=0)
    return df

def get_pos(name):
    path = pos_dir + '/' + name + '_selection.tsv'
    df = pd.read_csv(path, sep='\t')
    x = df['x'].values
    y = df['y'].values
    x = np.around(x).astype(int)
    y = np.around(y).astype(int)
    id = []
    for i in range(len(x)):
        id.append(str(x[i]) + 'x' + str(y[i]))
    df['id'] = id
    return df

def get_meta(name, gene_list=None):
    cnt = get_cnt(name)
    pos = get_pos(name)
    meta = cnt.join((pos.set_index('id')))
    return meta

def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)


### Calculate and save radius information


In [9]:
Processed_sample_dir = 'data/Processed_HER2/'
sample_name = 'H1'
new_sample_dir = os.path.join(Processed_sample_dir, sample_name)
if not os.path.exists(new_sample_dir):
    os.makedirs(new_sample_dir)


In [ ]:
img = get_img(sample_name)
radius = np.sqrt(np.prod(img.shape[:2]) / 32 / 34) / 4  # calculated by the 33 * 35 matrix

radius_file = open(f'{new_sample_dir}/{sample_name}_radius.txt', mode='w+')
radius_file.write(str(radius))

radius_file.close()

# save raw image
save_image(img,f'{new_sample_dir}/{sample_name}_raw.jpg')

In [44]:
save_image(img,f'{new_sample_dir}/{sample_name}_raw.jpg')

data/Processed_HER2/H1/H1_raw.jpg


## Process gene expression counts file and spatial coordinate positions file.

read data

In [10]:
cnt = get_cnt('H1')
pos = get_pos('H1')
# img = get_img('H1')

#### Save gene expression counts with spatial locations



In [12]:
cnt.index.name = 'spot'
cnt

,FO538757.1,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,C1orf159,...,CMC4,BRCC3,VBP1,RAB39B,CLIC2,TMLHE,SPRY3,VAMP7,UTY,TMSB4Y
spot,,,,,,,,,,,,,,,,,,,,,
10x10,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10x11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10x12,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10x13,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
10x14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9x29,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9x30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9x31,0,0,1,0,0,0,1,3,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
pos.set_index('id', inplace=True)
pos.index.name = 'spot'
pos

,x,y,new_x,new_y,pixel_x,pixel_y,selected
spot,,,,,,,
22x9,22,9,21.928,8.924,6082.85,2300.06,1
11x9,11,9,10.904,8.937,2878.66,2303.83,1
19x9,19,9,18.920,8.936,5208.56,2303.54,1
25x9,25,9,24.979,8.954,6969.65,2308.77,1
18x9,18,9,17.915,8.949,4916.45,2307.31,1
...,...,...,...,...,...,...,...
14x34,14,34,13.996,34.043,3777.37,9591.22,1
16x34,16,34,15.960,34.031,4348.22,9587.73,1
15x34,15,34,14.918,34.059,4045.35,9595.86,1


In [15]:
locs_main =  pos.reindex(cnt.index)
locs_main

,x,y,new_x,new_y,pixel_x,pixel_y,selected
spot,,,,,,,
10x10,10,10,9.880,9.969,2581.03,2603.38,1
10x11,10,11,9.890,10.978,2583.93,2896.26,1
10x12,10,12,9.881,11.939,2581.32,3175.21,1
10x13,10,13,9.890,12.937,2583.93,3464.89,1
10x14,10,14,9.880,13.954,2581.03,3760.09,1
...,...,...,...,...,...,...,...
9x29,9,29,8.976,29.048,2318.27,8141.34,1
9x30,9,30,8.973,30.019,2317.40,8423.19,1
9x31,9,31,8.979,31.034,2319.15,8717.81,1


In [39]:
type(locs)

pandas.core.frame.DataFrame

In [35]:
locs = locs_main.iloc[:,[4,5]].round().astype(int)
locs

data/Processed_HER2/H1/H1_locs.tsv


In [43]:
loc_file = open(f'{new_sample_dir}/{sample_name}_locs.txt',mode='a+')
for i in range(locs.shape[0]):
    x, y = locs.iloc[i]     
    loc_file.write(str(x)+','+str(y)+'\n')
loc_file.close()

In [ ]:
adata = ad.AnnData(X=cnt.astype(float))
adata.obsm['x'] = np.array(locs_main.iloc[:,4])
adata.obsm['y'] = np.array(locs_main.iloc[:,5])
adata.obsm['spatial'] = np.array(locs)
adata.uns['image'] = np.array(get_img('H1'))

In [29]:
adata.write(f'{new_sample_dir}/H1_whole_genes.h5ad')